In [ ]:
!pip install  git+https://github.com/himanshudabas/twint.git@origin/master#egg=twint
#gitインストール要

!pip install nlplot
!pip install janome
!pip install MeCab

In [9]:
import twint
import nest_asyncio

nest_asyncio.apply()

# Configure
c = twint.Config()
#特定のユーザー名
c.Username = "twitter"
#実行日から数えて過去何日分のツイートを取得するか
c.Limit = 30
#csv形式で保存するか
c.Store_csv = True
#保存ファイルの名前
#c.Output = "Sample.csv"

# Run
twint.run.Search(c)

1438167531164999687 2021-09-15 15:47:13 +0000 <Twitter> @imOKcozImJK don't be shocked if we scream about BTS in your DMs
1438165019263488008 2021-09-15 15:37:15 +0000 <Twitter> @NhatHoangRBLX on our way to wrestling Twitter
1438161789557035010 2021-09-15 15:24:25 +0000 <Twitter> @djarinculture first is the worst second is the best third is the one with the treasure chest
1438161655247089669 2021-09-15 15:23:53 +0000 <Twitter> @bre_naa following you because you're extraordinary
1438159860416253957 2021-09-15 15:16:45 +0000 <Twitter> @ImLiteralLarry1 you're also Larry
1438156066634747911 2021-09-15 15:01:40 +0000 <Twitter> @arikeokinn excited to be follower 543
1438153955117305860 2021-09-15 14:53:17 +0000 <Twitter> @hedasripa you could've just told us you'd put more carol danvers on our timeline
1438151307643260931 2021-09-15 14:42:45 +0000 <Twitter> @jakehue_ NEED this on our timeline
1438149367005843462 2021-09-15 14:35:03 +0000 <Twitter> @louanben where would we be without kissing-he

In [2]:
from datetime import datetime, timedelta, timezone

jst_tz = timezone(timedelta(hours=9), 'JST')

jh = timedelta(hours=9)
thb = timedelta(hours=1)
utc = datetime.now()
jst = utc + jh
before_1h = jst - thb
print(thb)
now_str = jst.strftime('%Y-%m-%d %H:%M')
before_1h_str = before_1h.strftime('%Y-%m-%d %H:%M')
print(now_str,before_1h_str)


1:00:00
2021-09-20 20:08 2021-09-20 19:08


In [4]:
import twint
import nest_asyncio
import pandas as pd
from datetime import datetime

#これないとエラーが出る・・・・
nest_asyncio.apply()

with open('user.txt', 'r') as f:
    user_list = f.read().split(",")

#hako_df = pd.DataFrame()
output_filename = "twint_scrape2.csv"

for i,twi in enumerate(user_list):
    try:
        c = twint.Config()
        c.Username = twi#解析するユーザー名
        #c.Store_json = True # Store_csvもあるがカラム名が入らなかったため不採用
        c.Store_csv = True
        c.Output = output_filename #上記アウトプットするJSONファイルの名前
        c.Since = before_1h_str#"2019-01-01 00:00:00" #ここから
        c.Until = now_str#"2021-09-01 00:00:00" #ここまで
        #c.Limit = 1
        #c.Pandas = True #出力形式をPandasにする
        c.Hide_output = False # False にするとツイートがターミナルに出力
        twint.run.Search(c)
    except ValueError:
        continue
        #df = twint.storage.panda.Tweets_df #データフレーム形式へ変換
        #Tweets_df = df['tweet'].to_frame() #tweetカラムのみデータフレーム形式で抽出
        #hako_df = pd.concat([hako_df, Tweets_df], axis=0)
'''
tweets_list = Tweets_df['tweet'].tolist() #tweetカラムをlist化して抽出しておく
a = list(Tweets_df.loc[: , "tweet"])
display(Tweets_df.loc[: , "tweet"])
'''

'\ntweets_list = Tweets_df[\'tweet\'].tolist() #tweetカラムをlist化して抽出しておく\na = list(Tweets_df.loc[: , "tweet"])\ndisplay(Tweets_df.loc[: , "tweet"])\n'

In [ ]:
import pandas as pd
df = pd.read_csv('twint_scrape.csv', encoding="utf-8")
Tweets_df = df.loc[: , "tweet"]
#display(Tweets_df)
with open('stop.txt', 'r') as f:
    stop_list = f.read().split(",")


In [ ]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import *
from janome.charfilter import *

#形態素解析器の設定
char_filters = [UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(r"[IiⅠｉ?.*/~=()〝 <>:：《°!！!？（）-]+", "")]
t = Tokenizer()
token_filters = [POSKeepFilter(["名詞"]), POSStopFilter(["名詞,非自立", "名詞,数", "名詞,代名詞", "名詞,接尾"]),LowerCaseFilter()]
analyzer = Analyzer(char_filters=char_filters, token_filters=token_filters)

Tweets_df = Tweets_df.apply(lambda x: " ".join([token.surface for token in analyzer.analyze(x)]))
Tweets_df = pd.DataFrame(Tweets_df)
text = ""
for i,data in enumerate(Tweets_df.loc[:,"tweet"]):
    text = text + data + ""

word_txt = {}
lines = text.split("\r\n")
for line in lines:
    malist = t.tokenize(line)
    for w in malist:
        word = w.surface#4---単語情報の読込
        ps = w.part_of_speech #5---品詞情報の読込
        if ps.find('名詞') < 0: continue #6---名詞のカウント
        if not word in word_txt:
            word_txt[word] = 0
        word_txt[word] += 1 #7---カウント
#8---頻出単語の表示
keys = sorted(word_txt.items(), key=lambda x:x[1], reverse=True)
print(type(keys))

for word,cnt in keys:
    if any([word == i for i in stop_list]):
        continue
    elif word == ",":
        continue
    else:
        print("{0}({1}) ".format(word,cnt), end="")
        print()

In [ ]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from wordcloud import STOPWORDS

for i,moji in enumerate(stop_list):
    STOPWORDS.add(moji)

for i,data in enumerate(Tweets_df.loc[:,"tweet"]):
    text = text + data + ""

stop = list(STOPWORDS)

In [ ]:
import twint
import csv
import os
import shutil
from datetime import datetime, timedelta
import functools
print = functools.partial(print, flush=True)
 
ORG_CSV_PATH = "to_csv_out.csv"
TMP_CSV_PATH = "tmp.csv"
 
def csv_merge():
 
    with open('test.csv','w') as n:
         f = open(ORG_CSV_PATH, 'r')
         reader = csv.reader(f)
         header = next(reader)
 
         m = open(TMP_CSV_PATH, 'r')
         tmp_reader = csv.reader(m)
         writer = csv.writer(n)
         for row in tmp_reader:
             writer.writerow(row)
         for row in reader:
             writer.writerow(row)
         m.close()
         f.close()
    shutil.move("test.csv", ORG_CSV_PATH)
 
def fetch_tweet(query):
 
    today = datetime.today()
    if (os.path.isfile(ORG_CSV_PATH)):
        since = today - timedelta(days=1)
    else:
        print("new csv")
        # 7 years
        since = today - timedelta(weeks=364)
 
    c = twint.Config()
    c.Search = query
    c.Store_csv = True
    c.Output = TMP_CSV_PATH
    c.Since = datetime.strftime(since, '%Y-%m-%d')
    twint.run.Search(c)
 
if __name__ == "__main__":
   fetch_tweet("uguisu.skr.jp -bot -recollection")
   csv_merge()

In [ ]:
#自然言語を簡単に可視化・分析できるライブラリ
import nlplot

npt = nlplot.NLPlot(Tweets_df, target_col='tweet')

stopwords = npt.get_stopword(top_n=10, min_freq=0)


In [ ]:
#N-gram bar chart
npt.bar_ngram(
    title='uni-gram',
    xaxis_label='word_count',
    yaxis_label='word',
    ngram=1,
    top_n=50,
    stopwords=stop,
)

In [ ]:
#N-gram tree Map
npt.treemap(
    title='Tree of Most Common Words',
    ngram=1,
    top_n=30,
    stopwords=stop,
)

In [ ]:
#Histogram of the word count
npt.word_distribution(
    title='number of words distribution',
    xaxis_label='count',
)

In [ ]:
npt.wordcloud(
    max_words=300,
    max_font_size=100,
    colormap='tab20_r',
    stopwords=stop,
)

In [ ]:
#co-occurrence networks
# ビルド（データ件数によっては処理に時間を要します）※ノードの数のみ変更
npt.build_graph(stopwords=stopwords, min_edge_frequency=1)

display(
    npt.node_df.head(), npt.node_df.shape,
    npt.edge_df.head(), npt.edge_df.shape
)

npt.co_network(
    title='Co-occurrence network',
)

In [ ]:
#display(Tweets_df)
#print(a)

text = " "
for i,txt in enumerate(a):
    text = text + " " + txt + "\n"

print(type(text))

from janome.tokenizer import Tokenizer

t = Tokenizer

for token in t.tokenize(text):
    print(token)

#wakati = [i for i in t.tokenize(text)]


In [ ]:
'''
import re
import MeCab

def word_extraction(text):
    """文を取得して特定名詞だけ抽出する関数"""

    #tagger = MeCab.Tagger('-d "./mecab-ipadic-neologd/neologd"')
    #邪魔なURL部分を正規表現で排除
    result = re.sub("https?://[\w!\?/\+\-_~=;\.,\*&@#\$%\(\)'\[\]]+", "", text)
    parse = MeCab.parse(result) #解析

    lines = parse.split('\n') #改行で区切る
    nounlist = [] #特定名詞を格納するリスト
    for line in lines:
        feature = line.split('\t') #タブで区切る
        if len(feature) == 2: #'EOS'と''を省く
            info = feature[1].split(',') #カンマで区切る
            hinshi = info[0] #その0番目(品詞)を取得
            hinshi_classi = info[1] #1番目(品詞分類)を取得
            #名詞 - 一般,サ変,固有名詞　だけ抽出する　※抽出したいものに合わせてカスタマイズすればいい
            if (hinshi in ('名詞')) and ((hinshi_classi in ('一般')) or (hinshi_classi in ('サ変接続')) or (hinshi_classi in ('固有名詞'))):
                nounlist.append(feature[0]) #info[6]を取得してもいいが、今回は原文を取得
    return nounlist

a = list(Tweets_df.loc[: , "tweet"])
word_extraction(text)
'''